In [1]:
import os
import numpy as np
import cv2

Questo codice crea un classificatore a cascata utilizzando l'algoritmo di Viola Jones per la rilevazione del volto. Viene utilizzato il file XML "haarcascade_frontalface_alt.xml" per fornire le informazioni necessarie per la rilevazione del volto. Il classificatore viene caricato utilizzando il metodo "load()" della classe CascadeClassifier di OpenCV. Infine, la funzione "viola_jones_init()" restituisce il classificatore e lo assegna alla variabile "face_cascade". Se ci sono problemi durante il caricamento del classificatore, viene stampato un messaggio di errore e il programma viene terminato.

In [2]:
def viola_jones_init():
    face_xml='haarcascade_frontalface_alt.xml'

    face_cascade=cv2.CascadeClassifier()

    if not face_cascade.load(face_xml):
        print('--(!)Error loading face cascade')
        exit(0)

    return face_cascade

face_cascade=viola_jones_init()

creo un dataset contenente le immagini di volti.
per ogni immagine tramite l'lgoritmo di Viola Jones rilevo la presenza di almeno un volto, se il volto viene rilevato questo viene riagliato e ridimensionato a 64X64.
Salvo i volti trovati in una lista "images" per poi convertirla in una matrice.
La matrice viene quindi trasposta per ottenere un array dove ogni riga rappresenta un'immagine e ogni colonna rappresenta il valore di un singolo pixel.

In [3]:
def initialize_dataset(face_cascade, pathname):
    images=[]

    i=0
    for dirpath, dirnames, filenames in  os.walk(pathname):
        for filename in filenames:
            img=cv2.imread(os.path.join(dirpath, filename))
            img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces=face_cascade.detectMultiScale(img_gray)

            for x,y,w,h in faces:
                face=img_gray[y:y+h, x:x+w]

            face_resized=cv2.resize(face, (64,64))
            images.append(face_resized.flatten())
            i+=1

            if i>2000:
                break

        if i>2000:
                break
    #creo la matrice trasposta
    images=np.array(images).T
    return images

Calcolo la faccia media per dataset di volti trovati

In [4]:
images=initialize_dataset(face_cascade, 'IMMAGINI')

mean=np.mean(images, axis=1)

mean_face=mean.reshape((64,64)).astype(np.uint8)

cv2.imshow("faccia media", mean_face)
cv2.imwrite("faccia media.png", mean_face)
cv2.waitKey(0)
cv2.destroyAllWindows()

Dalla matrice trasposta (images) trovata in precendenza mi calcolo la matrice di covarianza e con il metodo svd mi calcolo le tre matrici associate

In [5]:
def compute_eigenfaces(images):
    #calcola la matrice di covarianza delle immagini nella lista `images`
    covar=np.cov(images)
    #scompongo la matrice di covarianza nei tre componenti U autovettori, S autovalori, V
    U,S,V=np.linalg.svd(covar)
    # ritorno le matrici `U` (le "eigenfaces") e i rispettivi valori singolari `S`
    return U, S

eigenfaces, S=compute_eigenfaces(images)

Dal video "validation_1.mp4" creato di proposito, mi ricavo i vari volti con Viola Jones per addestrare l'algoritmo knn

In [6]:
def detect_face(face_cascade):

    images=[]

    video=cv2.VideoCapture("validation_1.mp4")

    if video is None or not video.isOpened():
        print('--(!)Error opening video capture')
        exit(0)

    ret, img = video.read()
    i=1

    while ret:

            img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces=face_cascade.detectMultiScale(img_gray)

            for x,y,w,h in faces:

                face=img_gray[y:y+h, x:x+w]

                face_resized=cv2.resize(face, (64,64))

                cv2.imwrite("validation/validate"+str(i)+".png", face_resized)
                i+=1

                images.append(face_resized.flatten())

            ret, img = video.read()

            if i>100:
                  break
    #ricalcolo la matrice di covarianza contenente i volti
    images=np.array(images).T
    return images

Lo scopo di matrix_A  è quello di calcolare le distanze tra le immagini e identificare l'immagine di test corrispondente a quella di training più vicina.

La funzione prende in input il numero di componenti principali "n_components" (eigenfaces) da utilizzare, la matrice di training "gallery" contenente le immagini della galleria e la matrice "eigenfaces" contenente gli autovettori della matrice di covarianza, crea una matrice "matrix_a" di dimensioni (n_images, n_components) e la popola con i valori della proiezione delle immagini nella galleria "gallery" sulla base delle prime "n_components" eigenfaces calcolate in precedenza.
Per ogni immagine nella galleria, la funzione calcola le componenti principali della proiezione nel nuovo spazio delle eigenfaces
Lo scopo della funzione è di creare la matrice "A", che rappresenta la proiezione di ciascuna immagine della galleria sullo spazio delle "eigenfaces"


In [7]:
def matrix_A(n_components, gallery, eigenfaces):

    n_images=gallery.shape[1]
    #dimensione matrice A
    matrix_a=np.zeros((n_images, n_components))

    for i in range(n_images):
        for j in range(n_components):
            #prodotto scalare tra il vettore dell'immagine meno la media delle img e ciascun autovettore in "eigenfaces"
            matrix_a[i,j]=np.dot(gallery[:,i]-mean, eigenfaces[:,j])

    return matrix_a

In [8]:
from sklearn.neighbors import KNeighborsClassifier as KNN
#"gallery" contenente immagini di volti noti
gallery=initialize_dataset(face_cascade, 'galleria')
#"validation" contenente immagini di volti da riconoscere
validation=detect_face(face_cascade)

Lo scopo della funzione "min_index" è quello di determinare il valore più piccolo dell'indice  della matrice S, tale che la somma dei primi indici di "S" sia maggiore o uguale a "n".

In [9]:
def min_index(S, n):
    sum=0
    index=0
    for i in range(S.shape[0]):
        sum+=S[i]
        if sum>=n:
            index=i
            break

    return index

In [10]:
# Definisco il percorso della cartella contenente le foto dei volti
face_images_path = "galleria"

# Creo una lista vuota per le etichette dei volti
etichette_facce = []

# Creo una lista vuota per le immagini dei volti
face_images = []

# Loop attraverso le sottocartelle nella cartella dei volti
for subject in os.listdir(face_images_path):
    # Creo il percorso completo alla cartella del soggetto
    subject_path = os.path.join(face_images_path, subject)
    # Loop attraverso le immagini nella cartella del soggetto
    for image_name in os.listdir(subject_path):
            # Creo il percorso completo all'immagine
            image_path = os.path.join(subject_path, image_name)
            # Leggo l'immagine usando OpenCV
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            # Aggiungo l'immagine alla lista delle immagini dei volti
            face_images.append(image)
            # Aggiungo l'etichetta del volto alla lista delle etichette dei volti
            etichette_facce.append(str(subject))

# Convertiamo le liste in array NumPy per utilizzarle con scikit-learn
face_images = np.array(face_images)
etichette_facce = np.array(etichette_facce)

print("face labels",etichette_facce)

face labels ['alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'antonio' 'antonio' 'antonio' 'antonio'
 'antonio' 'antonio' 'antonio' 'antonio' 'antonio' 'antonio' 'antonio'
 'antonio' 'antonio' 'antonio' 'antonio' 'antonio' 'antonio' 'antonio'
 'daniele' 'daniele' 'daniele' 'daniele' 'daniele' 'daniele' 'daniele'
 'daniele' 'daniele' 'daniele' 'daniele' 'daniele' 'rosario' 'rosario'
 'rosario' 'rosario' 'rosario' 'rosario' 'rosario' 'rosario' 'rosario'
 'rosario' 'rosario' 'rosario' 'rosario' 'rosario' 'rosario']


In [11]:
validate=['antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','rosario','alessandro','antonio','alessandro','antonio','alessandro','daniele','antonio','daniele','alessandro','antonio','alessandro','daniele','antonio','alessandro','daniele','antonio','alessandro','daniele','antonio','alessandro','daniele','antonio','alessandro','daniele','antonio','alessandro','daniele','antonio','rosario','alessandro','daniele','antonio','rosario','alessandro','daniele','rosario','antonio','alessandro','daniele','antonio','rosario','alessandro','daniele','daniele']


Algoritmo KNN con varianza tra 95% ed il 99.99% e valore di K[1,3,5]
valuta l'accuratezza del classificatore KNN sulla base delle proiezioni delle immagini nella galleria con il numero minimo di "eigenfaces".

In [12]:
#normalizzo la matrice S per avere dei valori più facili da comparare
S_normalized=S/np.sum(S)
#faccio variare la varianza tra 95% ed il 99.99%
for n in [0.95, 0.99]:

        index=min_index(S_normalized, n)
        a_train= matrix_A(index, gallery, eigenfaces)
        a_val=matrix_A(index, validation, eigenfaces)

        for k in [1,3,5]:
            knn= KNN(n_neighbors=k)
            #addestro il dataset passando con fit il valori di a_trai generato dalle varie immagini della galleria e le etichette dei vari volti
            knn.fit(a_train, etichette_facce)
            #calcola l'array di predizioni corrispondenti alle etichette dei dati
            predicted=knn.predict(a_val)
            print("predicted",predicted)
            #il metodo score restituisce l'accuratezza media sui dati di test e sulle etichette date
            acc=knn.score(a_val, validate )
            print(f"Con varianza pari al {n*100}% e K={k} la percentuale di accuratezza: {acc:.5f}")

C:\Users\admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

predicted ['alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'daniele'
 'alessandro' 'alessandro' 'alessandro' 'alessandro' 'alessandro'
 'daniele' 'alessandro' 'daniele' 'alessandro' 'alessandro' 'alessandro'
 'daniele' 'alessandro' 'alessandro' 'daniele' 'alessandro' 'a

C:\Users\admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

calcolo le prime 10 Eigenfaces
"Eigenfaces_img" prende in input due parametri: "matrix_a" ed "eigenfaces".
 Con un ciclo for itera sulle prime 10 immagini della matrice "matrix_a" e sulle prime 10 colonne dell'array "eigenfaces".

In [13]:
def eigenface_img(matrix_a, eigenfaces):
    sum=0
    i=0
    #assegno a n_images,n_components lo stesso numero di colonne della matrice_a
    n_images,n_components=matrix_a.shape

    for i in range(n_images)[0:10]:
        for j in range(n_components)[0:10]:
            #sommo alla variabile sum il prodotto tra il valore di "matrix_a" alla posizione [i,j] e la colonna j-esima dell'array "eigenfaces"
            sum+=matrix_a[i,j]*eigenfaces[:,j]

        new=(mean+sum).reshape((64,64)).astype(np.uint8)
        cv2.imshow("img", new)
        cv2.imwrite("Eigenfaces/eigenface{"+str(i)+"}.png", new)
        i=+1
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        sum=0
eigenface_img(a_val, eigenfaces)

Creo un dataset contenente le frame del video "test" .
Per ogni immagine tramite l'lgoritmo di Viola Jones rilevo la presenza di almeno un volto, se il volto viene rilevato questo viene riagliato e ridimensionato a 64X64.
Salvo i volti trovati in una lista "images" per poi convertirla in una matrice.
La matrice viene quindi trasposta per ottenere un array dove ogni riga rappresenta un'immagine e ogni colonna rappresenta il valore di un singolo pixel.

In [14]:
def test_dataset(face_cascade, pathname, filename):
    images=[]

    img=cv2.imread(os.path.join(pathname, filename))
    img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(img_gray)

    for x,y,w,h in faces:
        face=img_gray[y:y+h, x:x+w]
        face_resized=cv2.resize(face, (64,64))
        images.append(face_resized.flatten())


    images=np.array(images).T
    print(f"Analizzato il file: {filename}")

    cv2.imshow('frame',img)

    return images

Lo scopo di matrix_A_test è quello di creare la matrice di proiezione per le immagini di test nella galleria delle immagini di training.

La funzione prende in input il numero di componenti principali "n_components" (eigenfaces) da utilizzare, la matrice di training "gallery1" contenente le immagini della galleria e la matrice "eigenfaces" contenente gli autovettori della matrice di covarianza. Successivamente, la funzione crea una matrice "a_test" di dimensioni (n_images, n_components) e la popola con i valori della proiezione delle immagini di test nella galleria "gallery1" sulle prime "n_components" eigenfaces calcolate in precedenza.
Per ogni immagine di test nella galleria, la funzione calcola le componenti principali della proiezione nel nuovo spazio delle eigenfaces. Lo scopo della funzione è di creare la matrice "a_test", che rappresenta la proiezione di ciascuna immagine di test della galleria sullo spazio delle "eigenfaces".

In [15]:
def matrix_A_test(n_components, gallery1, eigenfaces):

    n_images=gallery1.shape[1]

    a_test=np.zeros((n_images, n_components))

    for i in range(n_images):
        for j in range(n_components):
            a_test[i,j]=np.dot(gallery1[:,i]-mean, eigenfaces[:,j])

    return a_test

Considero un singlo frame e tramite l'lagoritmo knn addestrato precedentemente con il metodo "predict" predico le identità dei volti presenti nel frame

In [16]:
from sklearn.neighbors import KNeighborsClassifier as KNN

frame=test_dataset(face_cascade, 'frame','frame543.png')
n = 0.99
index=min_index(S_normalized, n)
a_val= matrix_A_test(index, frame, eigenfaces)

for k in [1]:
        knn= KNN(n_neighbors=k)
        knn.fit(a_train, etichette_facce)
        predicted=knn.predict(a_val)
        print("il suo predicted è:",predicted)

cv2.waitKey(0)
cv2.destroyAllWindows()

Analizzato il file: frame543.png


C:\Users\admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


il suo predicted è: ['daniele' 'antonio' 'alessandro']


Analizzo il video "test.mp4" craeto in precedenza e per ogni frame tramite Viola Jones rilevo la presenza di almeno un volto

In [18]:
def video_detection():
    frames=[]
    # apre il video
    video = cv2.VideoCapture('test.mp4')

# controlla se il video è aperto correttamente
    if not video.isOpened():
        print('Errore nell\'apertura del video')
        exit()

    # legge i frame del video
    while True:
        ret, frame = video.read()

        # se il frame non viene letto correttamente, esce dal ciclo
        if not ret:
            break

        # rileva le facce nel frame
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

        # disegna un rettangolo intorno a ogni volto rilevato
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 4)

        # mostra il video
        cv2.imshow('Video', frame)

        #cv2.imwrite("Frame/frame"+str(i)+".png", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
# rilascia le risorse
    video.release()
    #out.release()
    cv2.destroyAllWindows()
video_detection()

analizzo il video "test.mp4" utilizzando il classificatore di Viola Jones per rilevare le facce in ogni frame. Successivamente, disegna un rettangolo intorno a ogni faccia rilevata e scrive il video di output con i rettangoli intorno alle facce rilevate in un nuovo file ('video_finale.mp4')

In [18]:
def video_test():
    # Aprire il video di input
    video = cv2.VideoCapture('Assigmnet 2/test.mp4')

    # Ottenere le informazioni del video
    fps = int(video.get(cv2.CAP_PROP_FPS))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Creare il video di output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output = cv2.VideoWriter('video_finale.mp4', fourcc, fps, (width, height))

    # Ciclo while per analizzare ogni frame del video
    while True:
        # Leggere un nuovo frame dal video di input
        ret, frame = video.read()
        # rileva le facce nel frame
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

        # disegna un rettangolo intorno a ogni volto rilevato
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 4)


        # Scrivere il frame con il risultato nel video di output
        output.write(frame)

        # Uscire dal ciclo se non ci sono più frame da leggere
        if not ret:
            break

    # Rilasciare le risorse
    video.release()
    output.release()
    cv2.destroyAllWindows()
video_test()